In [1]:
using JuMP
using GLPK
using CPUTime
using Statistics

In [2]:
model = Model(GLPK.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [3]:
# number of nurses
N = 15
# periods to schedule, number of days
T = 7
# number of shifts = 3

# standard number of hours by contract per nurse
k=floor(T/7)
H_base = 36
H = [H_base*k for item in 1:N]
# maximum number of hours for each nurse
H_max = 60*k

# number of nurses required per shift and duration of each shift
R = [6; 5; 4]
h = [7; 8; 9]

# list of nurses that on the last day of the previous period covered  the night shift
p_list= [1] #can be adjusted differently
# array with the values of p per each nurse and update the dictionary with p_list
p = zeros(N)
for item in p_list
    p[item]=1
end

# wages
w = ones(N) #can be adjusted differently

# rho must be greater than max(w[i]*h[s]) for all i and all s
rho = maximum(w*h')+1

# x_4_T is assumed to be zero for each nurse
x_4_T = zeros(N)

15-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [4]:
# create the variables
@variable(model, x[1:N, 1:3, 1:T], Bin)
@variable(model, z[1:N, 1:3, 1:T], Bin)
@variable(model, q[1:N, 1:3, 1:T], Bin)
@variable(model, alpha[1:3, 1:T])


3×7 Matrix{VariableRef}:
 alpha[1,1]  alpha[1,2]  alpha[1,3]  …  alpha[1,5]  alpha[1,6]  alpha[1,7]
 alpha[2,1]  alpha[2,2]  alpha[2,3]     alpha[2,5]  alpha[2,6]  alpha[2,7]
 alpha[3,1]  alpha[3,2]  alpha[3,3]     alpha[3,5]  alpha[3,6]  alpha[3,7]

In [5]:
###is_binary.(alpha)
###delete.(model, c_1)
###unregister.(model, C_1)

In [6]:
c=0.5
@objective(model, Min, sum((x[i,s,t]+c*z[i,s,t]+(1-c)*q[i,s,t])*h[s]*w[i] for i in 1:N for s in 1:3 for t in 1:T) 
    + rho*sum(alpha[s,t] for s in 1:3 for t in 1:T))


7 x[1,1,1] + 3.5 z[1,1,1] + 3.5 q[1,1,1] + 7 x[1,1,2] + 3.5 z[1,1,2] + 3.5 q[1,1,2] + 7 x[1,1,3] + 3.5 z[1,1,3] + 3.5 q[1,1,3] + 7 x[1,1,4] + 3.5 z[1,1,4] + 3.5 q[1,1,4] + 7 x[1,1,5] + 3.5 z[1,1,5] + 3.5 q[1,1,5] + 7 x[1,1,6] + 3.5 z[1,1,6] + 3.5 q[1,1,6] + 7 x[1,1,7] + 3.5 z[1,1,7] + 3.5 q[1,1,7] + 8 x[1,2,1] + 4 z[1,2,1] + 4 q[1,2,1] + 8 x[1,2,2] + 4 z[1,2,2] + 4 q[1,2,2] + 8 x[1,2,3] + 4 z[1,2,3] + 4 q[1,2,3] + 8 x[1,2,4] + 4 z[1,2,4] + 4 q[1,2,4] + 8 x[1,2,5] + 4 z[1,2,5] + 4 q[1,2,5] + 8 x[1,2,6] + 4 z[1,2,6] + 4 q[1,2,6] + 8 x[1,2,7] + 4 z[1,2,7] + 4 q[1,2,7] + 9 x[1,3,1] + 4.5 z[1,3,1] + 4.5 q[1,3,1] + 9 x[1,3,2] + 4.5 z[1,3,2] + 4.5 q[1,3,2] + 9 x[1,3,3] + 4.5 z[1,3,3] + 4.5 q[1,3,3] + 9 x[1,3,4] + 4.5 z[1,3,4] + 4.5 q[1,3,4] + 9 x[1,3,5] + 4.5 z[1,3,5] + 4.5 q[1,3,5] + 9 x[1,3,6] + 4.5 z[1,3,6] + 4.5 q[1,3,6] + 9 x[1,3,7] + 4.5 z[1,3,7] + 4.5 q[1,3,7] + 7 x[2,1,1] + 3.5 z[2,1,1] + 3.5 q[2,1,1] + 7 x[2,1,2] + 3.5 z[2,1,2] + 3.5 q[2,1,2] + 7 x[2,1,3] + 3.5 z[2,1,3] + 3.5 q[2,1,3

In [7]:
#constraints
@constraint(model, a_6, sum(x[:,s,:] for s in 1:3).<=1 )

15×7 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 x[1,1,1] + x[1,2,1] + x[1,3,1] <= 1.0     …  x[1,1,7] + x[1,2,7] + x[1,3,7] <= 1.0
 x[2,1,1] + x[2,2,1] + x[2,3,1] <= 1.0        x[2,1,7] + x[2,2,7] + x[2,3,7] <= 1.0
 x[3,1,1] + x[3,2,1] + x[3,3,1] <= 1.0        x[3,1,7] + x[3,2,7] + x[3,3,7] <= 1.0
 x[4,1,1] + x[4,2,1] + x[4,3,1] <= 1.0        x[4,1,7] + x[4,2,7] + x[4,3,7] <= 1.0
 x[5,1,1] + x[5,2,1] + x[5,3,1] <= 1.0        x[5,1,7] + x[5,2,7] + x[5,3,7] <= 1.0
 x[6,1,1] + x[6,2,1] + x[6,3,1] <= 1.0     …  x[6,1,7] + x[6,2,7] + x[6,3,7] <= 1.0
 x[7,1,1] + x[7,2,1] + x[7,3,1] <= 1.0        x[7,1,7] + x[7,2,7] + x[7,3,7] <= 1.0
 x[8,1,1] + x[8,2,1] + x[8,3,1] <= 1.0        x[8,1,7] + x[8,2,7] + x[8,3,7] <= 1.0
 x[9,1,1] + x[9,2,1] + x[9,3,1] <= 1.0        x[9,1,7] + x[9,2,7] + x[9,3,7] <= 1.0
 x[10,1,1] + x[10,2,1] + x[10,3,1] <= 1.0     x[10,1,7] + x[10,2,7] + x[10,3,7

In [8]:
@constraint(model, b_6, sum(x[i,:,:]+z[i,:,:] for i in 1:N) + alpha[:,:] .== R*(ones(T))')

3×7 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 x[1,1,1] + x[2,1,1] + x[3,1,1] + x[4,1,1] + x[5,1,1] + x[6,1,1] + x[7,1,1] + x[8,1,1] + x[9,1,1] + x[10,1,1] + x[11,1,1] + x[12,1,1] + x[13,1,1] + x[14,1,1] + x[15,1,1] + z[1,1,1] + z[2,1,1] + z[3,1,1] + z[4,1,1] + z[5,1,1] + z[6,1,1] + z[7,1,1] + z[8,1,1] + z[9,1,1] + z[10,1,1] + z[11,1,1] + z[12,1,1] + z[13,1,1] + z[14,1,1] + z[15,1,1] + alpha[1,1] == 6.0  …  x[1,1,7] + x[2,1,7] + x[3,1,7] + x[4,1,7] + x[5,1,7] + x[6,1,7] + x[7,1,7] + x[8,1,7] + x[9,1,7] + x[10,1,7] + x[11,1,7] + x[12,1,7] + x[13,1,7] + x[14,1,7] + x[15,1,7] + z[1,1,7] + z[2,1,7] + z[3,1,7] + z[4,1,7] + z[5,1,7] + z[6,1,7] + z[7,1,7] + z[8,1,7] + z[9,1,7] + z[10,1,7] + z[11,1,7] + z[12,1,7] + z[13,1,7] + z[14,1,7] + z[15,1,7] + alpha[1,7] == 6.0
 x[1,2,1] + x[2,2,1] + x[3,2,1] + x[4,2,1] + x[5,2,1] + x[6,2,1] + x[7,2,1] + x[8,2,1] + x[9,2,1] + x[10,2,1] +

In [9]:
@constraint(model, c_6, sum(x[:,s,t]*h[s] for s in 1:3 for t in 1:T).>= H)

15-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 7 x[1,1,1] + 8 x[1,2,1] + 9 x[1,3,1] + 7 x[1,1,2] + 8 x[1,2,2] + 9 x[1,3,2] + 7 x[1,1,3] + 8 x[1,2,3] + 9 x[1,3,3] + 7 x[1,1,4] + 8 x[1,2,4] + 9 x[1,3,4] + 7 x[1,1,5] + 8 x[1,2,5] + 9 x[1,3,5] + 7 x[1,1,6] + 8 x[1,2,6] + 9 x[1,3,6] + 7 x[1,1,7] + 8 x[1,2,7] + 9 x[1,3,7] >= 36.0
 7 x[2,1,1] + 8 x[2,2,1] + 9 x[2,3,1] + 7 x[2,1,2] + 8 x[2,2,2] + 9 x[2,3,2] + 7 x[2,1,3] + 8 x[2,2,3] + 9 x[2,3,3] + 7 x[2,1,4] + 8 x[2,2,4] + 9 x[2,3,4] + 7 x[2,1,5] + 8 x[2,2,5] + 9 x[2,3,5] + 7 x[2,1,6] + 8 x[2,2,6] + 9 x[2,3,6] + 7 x[2,1,7] + 8 x[2,2,7] + 9 x[2,3,7] >= 36.0
 7 x[3,1,1] + 8 x[3,2,1] + 9 x[3,3,1] + 7 x[3,1,2] + 8 x[3,2,2] + 9 x[3,3,2] + 7 x[3,1,3] + 8 x[3,2,3] + 9 x[3,3,3] + 7 x[3,1,4] + 8 x[3,2,4] + 9 x[3,3,4] + 7 x[3,1,5] + 8 x[3,2,5] + 9 x[3,3,5] + 7 x[3,1,6] + 8 x[3,2,6] + 9 x[3,3,6] + 7 x[3,1,7] + 8 x[3,2,7] + 9 x[

In [10]:
@constraint(model, j_6, sum((x[:,s,t]+c*z[:,s,t]+(1-c)*q[:,s,t])*h[s] for s in 1:3 for t in 1:T).<= H_max)


15-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 7 x[1,1,1] + 8 x[1,2,1] + 9 x[1,3,1] + 7 x[1,1,2] + 8 x[1,2,2] + 9 x[1,3,2] + 7 x[1,1,3] + 8 x[1,2,3] + 9 x[1,3,3] + 7 x[1,1,4] + 8 x[1,2,4] + 9 x[1,3,4] + 7 x[1,1,5] + 8 x[1,2,5] + 9 x[1,3,5] + 7 x[1,1,6] + 8 x[1,2,6] + 9 x[1,3,6] + 7 x[1,1,7] + 8 x[1,2,7] + 9 x[1,3,7] + 3.5 z[1,1,1] + 4 z[1,2,1] + 4.5 z[1,3,1] + 3.5 z[1,1,2] + 4 z[1,2,2] + 4.5 z[1,3,2] + 3.5 z[1,1,3] + 4 z[1,2,3] + 4.5 z[1,3,3] + 3.5 z[1,1,4] + 4 z[1,2,4] + 4.5 z[1,3,4] + 3.5 z[1,1,5] + 4 z[1,2,5] + 4.5 z[1,3,5] + 3.5 z[1,1,6] + 4 z[1,2,6] + 4.5 z[1,3,6] + 3.5 z[1,1,7] + 4 z[1,2,7] + 4.5 z[1,3,7] + 3.5 q[1,1,1] + 4 q[1,2,1] + 4.5 q[1,3,1] + 3.5 q[1,1,2] + 4 q[1,2,2] + 4.5 q[1,3,2] + 3.5 q[1,1,3] + 4 q[1,2,3] + 4.5 q[1,3,3] + 3.5 q[1,1,4] + 4 q[1,2,4] + 4.5 q[1,3,4] + 3.5 q[1,1,5] + 4 q[1,2,5] + 4.5 q[1,3,5] + 3.5 q[1,1,6] + 4 q[1,2,6] + 4.5 q[1,3,

In [11]:
@constraint(model, d_6, x[:, 3, 1:T-1] + sum(x[:,s,2:T] for s in 1:3).<= 1)


15×6 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 x[1,3,1] + x[1,1,2] + x[1,2,2] + x[1,3,2] <= 1.0      …  x[1,3,6] + x[1,1,7] + x[1,2,7] + x[1,3,7] <= 1.0
 x[2,3,1] + x[2,1,2] + x[2,2,2] + x[2,3,2] <= 1.0         x[2,3,6] + x[2,1,7] + x[2,2,7] + x[2,3,7] <= 1.0
 x[3,3,1] + x[3,1,2] + x[3,2,2] + x[3,3,2] <= 1.0         x[3,3,6] + x[3,1,7] + x[3,2,7] + x[3,3,7] <= 1.0
 x[4,3,1] + x[4,1,2] + x[4,2,2] + x[4,3,2] <= 1.0         x[4,3,6] + x[4,1,7] + x[4,2,7] + x[4,3,7] <= 1.0
 x[5,3,1] + x[5,1,2] + x[5,2,2] + x[5,3,2] <= 1.0         x[5,3,6] + x[5,1,7] + x[5,2,7] + x[5,3,7] <= 1.0
 x[6,3,1] + x[6,1,2] + x[6,2,2] + x[6,3,2] <= 1.0      …  x[6,3,6] + x[6,1,7] + x[6,2,7] + x[6,3,7] <= 1.0
 x[7,3,1] + x[7,1,2] + x[7,2,2] + x[7,3,2] <= 1.0         x[7,3,6] + x[7,1,7] + x[7,2,7] + x[7,3,7] <= 1.0
 x[8,3,1] + x[8,1,2] + x[8,2,2] + x[8,3,2] <= 1.0         x[8,3,6] + x[8,1,7] + x[8,2

In [12]:
@constraint(model, e_6, sum(x[:,s,1] for s in 1:3) .<= (1 .- p))


15-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 x[1,1,1] + x[1,2,1] + x[1,3,1] <= 0.0
 x[2,1,1] + x[2,2,1] + x[2,3,1] <= 1.0
 x[3,1,1] + x[3,2,1] + x[3,3,1] <= 1.0
 x[4,1,1] + x[4,2,1] + x[4,3,1] <= 1.0
 x[5,1,1] + x[5,2,1] + x[5,3,1] <= 1.0
 x[6,1,1] + x[6,2,1] + x[6,3,1] <= 1.0
 x[7,1,1] + x[7,2,1] + x[7,3,1] <= 1.0
 x[8,1,1] + x[8,2,1] + x[8,3,1] <= 1.0
 x[9,1,1] + x[9,2,1] + x[9,3,1] <= 1.0
 x[10,1,1] + x[10,2,1] + x[10,3,1] <= 1.0
 x[11,1,1] + x[11,2,1] + x[11,3,1] <= 1.0
 x[12,1,1] + x[12,2,1] + x[12,3,1] <= 1.0
 x[13,1,1] + x[13,2,1] + x[13,3,1] <= 1.0
 x[14,1,1] + x[14,2,1] + x[14,3,1] <= 1.0
 x[15,1,1] + x[15,2,1] + x[15,3,1] <= 1.0

In [13]:
@constraint(model, h_6, z[:,:,:] .<= 1 .- x[:,:,:])
@constraint(model, i_6, q[:,:,:] .<= 1 .- x[:,:,:])


15×3×7 Array{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}, 3}:
[:, :, 1] =
 x[1,1,1] + q[1,1,1] <= 1.0    …  x[1,3,1] + q[1,3,1] <= 1.0
 x[2,1,1] + q[2,1,1] <= 1.0       x[2,3,1] + q[2,3,1] <= 1.0
 x[3,1,1] + q[3,1,1] <= 1.0       x[3,3,1] + q[3,3,1] <= 1.0
 x[4,1,1] + q[4,1,1] <= 1.0       x[4,3,1] + q[4,3,1] <= 1.0
 x[5,1,1] + q[5,1,1] <= 1.0       x[5,3,1] + q[5,3,1] <= 1.0
 x[6,1,1] + q[6,1,1] <= 1.0    …  x[6,3,1] + q[6,3,1] <= 1.0
 x[7,1,1] + q[7,1,1] <= 1.0       x[7,3,1] + q[7,3,1] <= 1.0
 x[8,1,1] + q[8,1,1] <= 1.0       x[8,3,1] + q[8,3,1] <= 1.0
 x[9,1,1] + q[9,1,1] <= 1.0       x[9,3,1] + q[9,3,1] <= 1.0
 x[10,1,1] + q[10,1,1] <= 1.0     x[10,3,1] + q[10,3,1] <= 1.0
 x[11,1,1] + q[11,1,1] <= 1.0  …  x[11,3,1] + q[11,3,1] <= 1.0
 x[12,1,1] + q[12,1,1] <= 1.0     x[12,3,1] + q[12,3,1] <= 1.0
 x[13,1,1] + q[13,1,1] <= 1.0     x[13,3,1] + q[13,3,1] <= 1.0
 x[14,1,1] + q[14

In [14]:
@constraint(model, k_6, sum(z[:,s,:]+q[:,s,:] for s in 1:3) .<= 1)

15×7 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 z[1,1,1] + z[1,2,1] + z[1,3,1] + q[1,1,1] + q[1,2,1] + q[1,3,1] <= 1.0        …  z[1,1,7] + z[1,2,7] + z[1,3,7] + q[1,1,7] + q[1,2,7] + q[1,3,7] <= 1.0
 z[2,1,1] + z[2,2,1] + z[2,3,1] + q[2,1,1] + q[2,2,1] + q[2,3,1] <= 1.0           z[2,1,7] + z[2,2,7] + z[2,3,7] + q[2,1,7] + q[2,2,7] + q[2,3,7] <= 1.0
 z[3,1,1] + z[3,2,1] + z[3,3,1] + q[3,1,1] + q[3,2,1] + q[3,3,1] <= 1.0           z[3,1,7] + z[3,2,7] + z[3,3,7] + q[3,1,7] + q[3,2,7] + q[3,3,7] <= 1.0
 z[4,1,1] + z[4,2,1] + z[4,3,1] + q[4,1,1] + q[4,2,1] + q[4,3,1] <= 1.0           z[4,1,7] + z[4,2,7] + z[4,3,7] + q[4,1,7] + q[4,2,7] + q[4,3,7] <= 1.0
 z[5,1,1] + z[5,2,1] + z[5,3,1] + q[5,1,1] + q[5,2,1] + q[5,3,1] <= 1.0           z[5,1,7] + z[5,2,7] + z[5,3,7] + q[5,1,7] + q[5,2,7] + q[5,3,7] <= 1.0
 z[6,1,1] + z[6,2,1] + z[6,3,1] + q[6,1,1] + q[6,2,1] + q[6,3,1] <= 1

In [15]:
@constraint(model, l_6, sum(z[i,:,:] for i in 1:N) .== sum(q[i,:,:] for i in 1:N))

3×7 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 z[1,1,1] + z[2,1,1] + z[3,1,1] + z[4,1,1] + z[5,1,1] + z[6,1,1] + z[7,1,1] + z[8,1,1] + z[9,1,1] + z[10,1,1] + z[11,1,1] + z[12,1,1] + z[13,1,1] + z[14,1,1] + z[15,1,1] - q[1,1,1] - q[2,1,1] - q[3,1,1] - q[4,1,1] - q[5,1,1] - q[6,1,1] - q[7,1,1] - q[8,1,1] - q[9,1,1] - q[10,1,1] - q[11,1,1] - q[12,1,1] - q[13,1,1] - q[14,1,1] - q[15,1,1] == 0.0  …  z[1,1,7] + z[2,1,7] + z[3,1,7] + z[4,1,7] + z[5,1,7] + z[6,1,7] + z[7,1,7] + z[8,1,7] + z[9,1,7] + z[10,1,7] + z[11,1,7] + z[12,1,7] + z[13,1,7] + z[14,1,7] + z[15,1,7] - q[1,1,7] - q[2,1,7] - q[3,1,7] - q[4,1,7] - q[5,1,7] - q[6,1,7] - q[7,1,7] - q[8,1,7] - q[9,1,7] - q[10,1,7] - q[11,1,7] - q[12,1,7] - q[13,1,7] - q[14,1,7] - q[15,1,7] == 0.0
 z[1,2,1] + z[2,2,1] + z[3,2,1] + z[4,2,1] + z[5,2,1] + z[6,2,1] + z[7,2,1] + z[8,2,1] + z[9,2,1] + z[10,2,1] + z[11,2,1] + z[12,2,1] + z

In [16]:
@constraint(model, o_6, alpha[:,:] .>= 0)


3×7 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 alpha[1,1] >= 0.0  alpha[1,2] >= 0.0  …  alpha[1,7] >= 0.0
 alpha[2,1] >= 0.0  alpha[2,2] >= 0.0     alpha[2,7] >= 0.0
 alpha[3,1] >= 0.0  alpha[3,2] >= 0.0     alpha[3,7] >= 0.0

In [17]:
        
@constraint(model, f_6_1, z[:,1,1] .<= p)# for s=1 and t=1
@constraint(model, f_6_2, z[:,1,2:T] .<= x[:,3,1:T-1])# for s=1 and t!=1
@constraint(model, f_6_3, z[:,2:3,:] .<= x[:,1:2,:])#for s!=1

15×2×7 Array{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}, 3}:
[:, :, 1] =
 -x[1,1,1] + z[1,2,1] <= 0.0    -x[1,2,1] + z[1,3,1] <= 0.0
 -x[2,1,1] + z[2,2,1] <= 0.0    -x[2,2,1] + z[2,3,1] <= 0.0
 -x[3,1,1] + z[3,2,1] <= 0.0    -x[3,2,1] + z[3,3,1] <= 0.0
 -x[4,1,1] + z[4,2,1] <= 0.0    -x[4,2,1] + z[4,3,1] <= 0.0
 -x[5,1,1] + z[5,2,1] <= 0.0    -x[5,2,1] + z[5,3,1] <= 0.0
 -x[6,1,1] + z[6,2,1] <= 0.0    -x[6,2,1] + z[6,3,1] <= 0.0
 -x[7,1,1] + z[7,2,1] <= 0.0    -x[7,2,1] + z[7,3,1] <= 0.0
 -x[8,1,1] + z[8,2,1] <= 0.0    -x[8,2,1] + z[8,3,1] <= 0.0
 -x[9,1,1] + z[9,2,1] <= 0.0    -x[9,2,1] + z[9,3,1] <= 0.0
 -x[10,1,1] + z[10,2,1] <= 0.0  -x[10,2,1] + z[10,3,1] <= 0.0
 -x[11,1,1] + z[11,2,1] <= 0.0  -x[11,2,1] + z[11,3,1] <= 0.0
 -x[12,1,1] + z[12,2,1] <= 0.0  -x[12,2,1] + z[12,3,1] <= 0.0
 -x[13,1,1] + z[13,2,1] <= 0.0  -x[13,2,1] + z[13,3,1] <= 0.0
 -x[14,1,1] + z[14,2,1] <= 0.0

In [18]:

@constraint(model, g_6_1, q[:,3,T] .<= x_4_T)# for s=3 and t=T
@constraint(model, g_6_2, q[:,3,1:T-1] .<= x[:,1,2:T])# for s=3 and t!=T
@constraint(model, g_6_3, q[:,1:2,:] .<= x[:,2:3,:])# for s!=3




15×2×7 Array{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}, 3}:
[:, :, 1] =
 -x[1,2,1] + q[1,1,1] <= 0.0    -x[1,3,1] + q[1,2,1] <= 0.0
 -x[2,2,1] + q[2,1,1] <= 0.0    -x[2,3,1] + q[2,2,1] <= 0.0
 -x[3,2,1] + q[3,1,1] <= 0.0    -x[3,3,1] + q[3,2,1] <= 0.0
 -x[4,2,1] + q[4,1,1] <= 0.0    -x[4,3,1] + q[4,2,1] <= 0.0
 -x[5,2,1] + q[5,1,1] <= 0.0    -x[5,3,1] + q[5,2,1] <= 0.0
 -x[6,2,1] + q[6,1,1] <= 0.0    -x[6,3,1] + q[6,2,1] <= 0.0
 -x[7,2,1] + q[7,1,1] <= 0.0    -x[7,3,1] + q[7,2,1] <= 0.0
 -x[8,2,1] + q[8,1,1] <= 0.0    -x[8,3,1] + q[8,2,1] <= 0.0
 -x[9,2,1] + q[9,1,1] <= 0.0    -x[9,3,1] + q[9,2,1] <= 0.0
 -x[10,2,1] + q[10,1,1] <= 0.0  -x[10,3,1] + q[10,2,1] <= 0.0
 -x[11,2,1] + q[11,1,1] <= 0.0  -x[11,3,1] + q[11,2,1] <= 0.0
 -x[12,2,1] + q[12,1,1] <= 0.0  -x[12,3,1] + q[12,2,1] <= 0.0
 -x[13,2,1] + q[13,1,1] <= 0.0  -x[13,3,1] + q[13,2,1] <= 0.0
 -x[14,2,1] + q[14,1,1] <= 0.0

In [19]:
@constraint(model, m_6, z[:,3,1:T-1] + x[:,1,2:T] .<= 1)


15×6 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 x[1,1,2] + z[1,3,1] <= 1.0    …  x[1,1,7] + z[1,3,6] <= 1.0
 x[2,1,2] + z[2,3,1] <= 1.0       x[2,1,7] + z[2,3,6] <= 1.0
 x[3,1,2] + z[3,3,1] <= 1.0       x[3,1,7] + z[3,3,6] <= 1.0
 x[4,1,2] + z[4,3,1] <= 1.0       x[4,1,7] + z[4,3,6] <= 1.0
 x[5,1,2] + z[5,3,1] <= 1.0       x[5,1,7] + z[5,3,6] <= 1.0
 x[6,1,2] + z[6,3,1] <= 1.0    …  x[6,1,7] + z[6,3,6] <= 1.0
 x[7,1,2] + z[7,3,1] <= 1.0       x[7,1,7] + z[7,3,6] <= 1.0
 x[8,1,2] + z[8,3,1] <= 1.0       x[8,1,7] + z[8,3,6] <= 1.0
 x[9,1,2] + z[9,3,1] <= 1.0       x[9,1,7] + z[9,3,6] <= 1.0
 x[10,1,2] + z[10,3,1] <= 1.0     x[10,1,7] + z[10,3,6] <= 1.0
 x[11,1,2] + z[11,3,1] <= 1.0  …  x[11,1,7] + z[11,3,6] <= 1.0
 x[12,1,2] + z[12,3,1] <= 1.0     x[12,1,7] + z[12,3,6] <= 1.0
 x[13,1,2] + z[13,3,1] <= 1.0     x[13,1,7] + z[13,3,6] <= 1.0
 x[14,1,2] + z[14,3,1] <= 1.0    

In [20]:
@constraint(model, n_6, q[:,3,:] + x[:,2,:] .<= 1)

15×7 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 x[1,2,1] + q[1,3,1] <= 1.0    …  x[1,2,7] + q[1,3,7] <= 1.0
 x[2,2,1] + q[2,3,1] <= 1.0       x[2,2,7] + q[2,3,7] <= 1.0
 x[3,2,1] + q[3,3,1] <= 1.0       x[3,2,7] + q[3,3,7] <= 1.0
 x[4,2,1] + q[4,3,1] <= 1.0       x[4,2,7] + q[4,3,7] <= 1.0
 x[5,2,1] + q[5,3,1] <= 1.0       x[5,2,7] + q[5,3,7] <= 1.0
 x[6,2,1] + q[6,3,1] <= 1.0    …  x[6,2,7] + q[6,3,7] <= 1.0
 x[7,2,1] + q[7,3,1] <= 1.0       x[7,2,7] + q[7,3,7] <= 1.0
 x[8,2,1] + q[8,3,1] <= 1.0       x[8,2,7] + q[8,3,7] <= 1.0
 x[9,2,1] + q[9,3,1] <= 1.0       x[9,2,7] + q[9,3,7] <= 1.0
 x[10,2,1] + q[10,3,1] <= 1.0     x[10,2,7] + q[10,3,7] <= 1.0
 x[11,2,1] + q[11,3,1] <= 1.0  …  x[11,2,7] + q[11,3,7] <= 1.0
 x[12,2,1] + q[12,3,1] <= 1.0     x[12,2,7] + q[12,3,7] <= 1.0
 x[13,2,1] + q[13,3,1] <= 1.0     x[13,2,7] + q[13,3,7] <= 1.0
 x[14,2,1] + q[14,3,1] <= 1.0    

In [21]:
optimize!(model)

In [22]:
println()
println("The solver stopped because: ")
@show termination_status(model)# run it in ipynb file to get userfriendly result


The solver stopped because: 
termination_status(model) = MathOptInterface.OPTIMAL


OPTIMAL::TerminationStatusCode = 1

In [23]:
println()
println("to see if the solver found a primal feasible point:")
@show primal_status(model)# run it in ipynb file to get userfriendly result


to see if the solver found a primal feasible point:
primal_status(model) = MathOptInterface.FEASIBLE_POINT


FEASIBLE_POINT::ResultStatusCode = 1

In [24]:
println()
println("minimum cost:")
@show objective_value(model)# run it in ipynb file to get userfriendly result


minimum cost:
objective_value(model) = 826.0


826.0

In [25]:
println()
println("Matrix alpha is:")
@show value.(alpha)#run it in ipynb file to get userfriendly result


Matrix alpha is:
value.(alpha) = [0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0]


3×7 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [26]:
println()
println("max(alpha[s,t] for all s and all t) is: ", maximum(value.(alpha)))


max(alpha[s,t] for all s and all t) is: 0.0


In [27]:
println()
println("sum(alpha[s,t] for all s and all t) is: ", round(value.(sum(alpha))))


sum(alpha[s,t] for all s and all t) is: 0.0


In [28]:
fault_num=0
for s in 1:3
    for t in 1:T
        if value.(alpha[s,t]) >= 0.000001
            global fault_num += 1
        end
    end
end
println("number of cases in which the number of nurses is below the required quantity is: ", fault_num)

number of cases in which the number of nurses is below the required quantity is: 0


In [29]:
println()
println("CPU Time:")
CPU_Time = @time  optimize!(model)



CPU Time:
  3.448992 seconds (974 allocations: 110.188 KiB)


In [30]:
#number of regular shifts for each nurse:
println("-------------------------------------------")
for i in 1:N
    println("number of total shifts for nurse_", i, " is: ", value.(sum(x[i,s,t] for s in 1:3 for t in 1:T)))
end

-------------------------------------------
number of total shifts for nurse_1 is: 5.0
number of total shifts for nurse_2 is: 5.0
number of total shifts for nurse_3 is: 5.0
number of total shifts for nurse_4 is: 7.0
number of total shifts for nurse_5 is: 5.0
number of total shifts for nurse_6 is: 6.0
number of total shifts for nurse_7 is: 6.0
number of total shifts for nurse_8 is: 5.0
number of total shifts for nurse_9 is: 5.0
number of total shifts for nurse_10 is: 5.0
number of total shifts for nurse_11 is: 5.0
number of total shifts for nurse_12 is: 5.0
number of total shifts for nurse_13 is: 5.0
number of total shifts for nurse_14 is: 5.0
number of total shifts for nurse_15 is: 5.0


In [31]:
# number of extra shifts done by each nurse
println()
println("---------First c of shift (after regular shift)---------")
for i in 1:N
    println("nurse_", i, ": ", value.(sum(z[i,s,t] for s in 1:3 for t in 1:T)), " shifts")
end
println("---------Last 1-c hours of shift (before regular shift)---------")
for i in 1:N
    println("nurse_", i, ": ", value.(sum(q[i,s,t] for s in 1:3 for t in 1:T)), " shifts")
end


---------First c of shift (after regular shift)---------
nurse_1: 2.0 shifts
nurse_2: 1.0 shifts
nurse_3: 1.0 shifts
nurse_4: 1.0 shifts
nurse_5: 2.0 shifts
nurse_6: 1.0 shifts
nurse_7: 2.0 shifts
nurse_8: 0.0 shifts
nurse_9: 3.0 shifts
nurse_10: 2.0 shifts
nurse_11: 4.0 shifts
nurse_12: 3.0 shifts
nurse_13: 1.0 shifts
nurse_14: 2.0 shifts
nurse_15: 1.0 shifts
---------Last 1-c hours of shift (before regular shift)---------
nurse_1: 3.0 shifts
nurse_2: 2.0 shifts
nurse_3: 3.0 shifts
nurse_4: 1.0 shifts
nurse_5: 0.0 shifts
nurse_6: 2.0 shifts
nurse_7: 2.0 shifts
nurse_8: 1.0 shifts
nurse_9: 1.0 shifts
nurse_10: 2.0 shifts
nurse_11: 1.0 shifts
nurse_12: 2.0 shifts
nurse_13: 3.0 shifts
nurse_14: 1.0 shifts
nurse_15: 2.0 shifts


In [32]:
# Is the number of N nurses  enough to satisfy the demand?
println("-------------------------------------------")
if value.(sum(alpha)) >= 0.000001
    println("NO! The number of nurses is not enough to satisfy the demand")
else
    println("YES! The number of nurses is enough to satisfy the demand")
end


-------------------------------------------
YES! The number of nurses is enough to satisfy the demand


In [33]:
# How many hours does each nurse work in regular shifs?
println("-------------------------------------------")
for i in 1:N
    println("nurse_", i, " works for: ", value.(sum(x[i,s,t]*h[s] for s in 1:3 for t in 1:T)), " hours in regular shifs.")
end
# How many extra hours does each nurse work?
println("-------------------------------------------")
for i in 1:N
    println("nurse_", i, " works for: ", value.(sum((c*z[i,s,t]+(1-c)*q[i,s,t])*h[s] for s in 1:3 for t in 1:T)), " extra hours.")
end
println()
println("Maximum number of extra hours worked: ", maximum(value.(sum((c*z[:,s,t]+(1-c)*q[:,s,t])*h[s] for s in 1:3 for t in 1:T))))

-------------------------------------------
nurse_1 works for: 37.0 hours in regular shifs.
nurse_2 works for: 42.0 hours in regular shifs.
nurse_3 works for: 42.0 hours in regular shifs.
nurse_4 works for: 49.0 hours in regular shifs.
nurse_5 works for: 39.0 hours in regular shifs.
nurse_6 works for: 47.0 hours in regular shifs.
nurse_7 works for: 43.0 hours in regular shifs.
nurse_8 works for: 39.0 hours in regular shifs.
nurse_9 works for: 38.0 hours in regular shifs.
nurse_10 works for: 37.0 hours in regular shifs.
nurse_11 works for: 39.0 hours in regular shifs.
nurse_12 works for: 40.0 hours in regular shifs.
nurse_13 works for: 41.0 hours in regular shifs.
nurse_14 works for: 41.0 hours in regular shifs.
nurse_15 works for: 39.0 hours in regular shifs.
-------------------------------------------
nurse_1 works for: 20.5 extra hours.
nurse_2 works for: 12.5 extra hours.
nurse_3 works for: 15.0 extra hours.
nurse_4 works for: 8.5 extra hours.
nurse_5 works for: 8.5 extra hours.
nur

In [34]:
#Overall hours worked for each nurse
println("-------------------------------------------")
for i in 1:N
    println("nurse_", i, " works for: ", value.(sum((x[i,s,t]+c*z[i,s,t]+(1-c)*q[i,s,t])*h[s] for s in 1:3 for t in 1:T)), " hours in total.")
end

-------------------------------------------
nurse_1 works for: 57.5 hours in total.
nurse_2 works for: 54.5 hours in total.
nurse_3 works for: 57.0 hours in total.
nurse_4 works for: 57.5 hours in total.
nurse_5 works for: 47.5 hours in total.
nurse_6 works for: 58.5 hours in total.
nurse_7 works for: 60.0 hours in total.
nurse_8 works for: 42.5 hours in total.
nurse_9 works for: 55.5 hours in total.
nurse_10 works for: 54.0 hours in total.
nurse_11 works for: 59.0 hours in total.
nurse_12 works for: 60.0 hours in total.
nurse_13 works for: 57.0 hours in total.
nurse_14 works for: 53.0 hours in total.
nurse_15 works for: 52.5 hours in total.


In [35]:
#Maximum number of total hours worked
println()
println("Maximum number of total hours worked: ", maximum(value.(sum((x[:,s,t]+c*z[:,s,t]+(1-c)*q[:,s,t])*h[s] for s in 1:3 for t in 1:T))))


Maximum number of total hours worked: 60.0


In [ ]:
########The End!